In [2]:
! mkdir ~/.kaggle

In [3]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c plant-seedlings-classification
!unzip /content/plant-seedlings-classification.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open /content/plant-seedlings-classification.zip, /content/plant-seedlings-classification.zip.zip or /content/plant-seedlings-classification.zip.ZIP.


In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os

In [5]:
torch.cuda.empty_cache()

In [6]:
label = list(os.listdir('/content/train'))

FileNotFoundError: ignored

In [ ]:
#Loading Data
data_path_train = '/content/train'
data_path_test = '/content/test'
batch_size=16

resized_width=224
resized_height=224
classes = label

def load_split_train_test(datadir_train,datadir_test, valid_size = .2):
    train_transforms = transforms.Compose([
        transforms.Resize((resized_width,resized_height)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))               
        ])
    test_transforms = transforms.Compose([
        transforms.Resize((resized_width,resized_height)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))               
        ])
    train_data = torchvision.datasets.ImageFolder(
        root=datadir_train,  
        transform=train_transforms
    )
    test_data = torchvision.datasets.ImageFolder(
        root=datadir_test,
        transform=test_transforms
    )
    num_train = len(train_data)
    num_test =len(test_data)
    indices_train = list(range(num_train))
    indices_test = list(range(num_test))
    #split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices_train)
    from torch.utils.data.sampler import SubsetRandomSampler
    
    
    train_sampler = SubsetRandomSampler(indices_train)
    test_sampler = SubsetRandomSampler(indices_test)
    trainloader = torch.utils.data.DataLoader(
            train_data,
            batch_size=batch_size,
            num_workers=4,
#             shuffle=True,                                # Mutually exclusive with sampler
            sampler=train_sampler
    )
    testloader = torch.utils.data.DataLoader(
            test_data,
            batch_size=batch_size,
            num_workers=4,
#             shuffle=True,                               # Mutually exclusive with sampler
            sampler=test_sampler
    )
    return trainloader, testloader

trainloader, testloader = load_split_train_test(datadir_train=data_path_train,datadir_test=data_path_test, valid_size = .2)

In [ ]:
import torch.nn as nn

In [ ]:
class CNN_model(nn.Module):
  def __init__(self,input_shape,output_shape):
    super().__init__()
    
    self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,out_channels =64,kernel_size=3,
                      stride =1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,out_channels =128,kernel_size=3,
                      stride =1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
    )

    self.block2 = nn.Sequential(
        nn.Conv2d(in_channels=128,out_channels =128,kernel_size=3,
                      stride =1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128,out_channels =128,kernel_size=3,
                      stride =1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
    )
    self.block3 = nn.Sequential(
        nn.Conv2d(in_channels=128,out_channels =128,kernel_size=3,
                      stride =1,padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128,out_channels =64,kernel_size=3,
                      stride =1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
    )
    self.classifier = nn.Sequential(
      nn.Flatten(start_dim=1),
      nn.Linear(in_features = 28*28*64, out_features=96),
      #nn.Flatten(),
      nn.Linear(in_features = 96, out_features=12)

      
    )
    

  def forward(self,x):
    x=self.block1(x)
    #print(x.shape)
    #print(56*56*self.hu)
    x = self.block2(x)
    x = self.block3(x)
    #print(x.shape)
    x=self.classifier(x)
    #print(x.shape)
    return x

In [ ]:
CNN = CNN_model(input_shape =3,output_shape=len(label))

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
CNN=CNN.to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = CNN.parameters(),lr =0.1)

In [ ]:
!pip install torchmetrics

In [ ]:
import torchmetrics

In [ ]:
accuracy = torchmetrics.Accuracy()
accuracy = accuracy.to(device)

In [ ]:
a=next(iter(trainloader))

In [ ]:
a[0].shape

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
Epochs = 8

for epoch in range(Epochs):
  train_loss,train_acc =0,0
  print(f"------Epoch:{epoch}")
  for btach,(X,y) in enumerate(trainloader):
    X,y = X.to(device),y.to(device)
    y_pred = CNN(X)

    loss = loss_fn(y_pred,y)
    acc=accuracy(y_pred.argmax(dim=1), y)
    train_loss += loss
    train_acc += acc
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss /= len(trainloader)
  train_acc /= len(trainloader)
  print("************************")
  print(f"  set loss :  {train_loss}")
  print(f"set accuracy : {train_acc}")



In [ ]:
from torchvision.models import EfficientNet_B0_Weights

In [ ]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT 
weights

In [ ]:
auto_transforms = weights.transforms()
auto_transforms

In [ ]:
batch_size=32

def load_split_train_test(datadir_train,transform, valid_size = .2):
    train_transforms = transform
    test_transforms = transform
    train_data = torchvision.datasets.ImageFolder(
        root=datadir_train,  
        transform=train_transforms
    )
    
    num_train = len(train_data)
    
    indices_train = list(range(num_train))
    
    #split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices_train)
    from torch.utils.data.sampler import SubsetRandomSampler
    
    
    train_sampler = SubsetRandomSampler(indices_train)
    
    trainloader = torch.utils.data.DataLoader(
            train_data,
            batch_size=batch_size,
            num_workers=4,
#             shuffle=True,                                # Mutually exclusive with sampler
            sampler=train_sampler
    )
    
    return trainloader

train_loader = load_split_train_test(datadir_train=data_path_train,
                                                transform=auto_transforms, valid_size = .2)

In [ ]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary

In [ ]:
summary(model=model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20)
      

In [ ]:
for params in model.features.parameters():
  params.requires_grad=False

In [ ]:
model.classifier

In [ ]:
output_shape =12
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features = 1280, out_features =12,bias=True)
).to(device)

In [ ]:
model =model.to(device)

In [ ]:
summary(model=model, 
        input_size=(32, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
a=next(iter(train_loader))

In [ ]:
model(a[0].to(device)).argmax(dim=1)

In [ ]:
Epochs=10
for epoch in range(Epochs):
  train_loss,train_acc =0,0
  print(f"------Epoch:{epoch}")
  for btach,(X,y) in enumerate(train_loader):
    X,y = X.to(device),y.to(device)
    y_pred = model(X)

    loss = loss_func(y_pred,y)
    acc=accuracy(y_pred.argmax(dim=1), y)
    train_loss += loss
    train_acc += acc
    
    optim.zero_grad()
    loss.backward()
    optim.step()

  train_loss /= len(train_loader)
  train_acc /= len(train_loader)
  print(y_pred.argmax(dim=1))
  print(y)
  print("************************")
  print(f"  set loss :  {train_loss}")
  print(f"set accuracy : {train_acc}")

In [ ]:
test_transform = auto_transforms
test_data = torchvision.datasets.ImageFolder(
        root=data_path_test,  
        transform= test_transform
    )
traindata_loader = torchvision.datasets.dataloader(
    test_data
)